In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel(r'上證指數.xlsx')

In [ ]:
df

,名稱,代碼,日期,開盤,最高,最低,收盤,漲跌
0,上證指數,980001,2017-01-03,3105.31,3136.46,3105.31,3135.92,1
1,上證指數,980001,2017-01-04,3133.79,3160.10,3130.11,3158.79,1
2,上證指數,980001,2017-01-05,3157.91,3168.50,3154.28,3165.41,1
3,上證指數,980001,2017-01-06,3163.78,3172.03,3153.03,3154.32,0
4,上證指數,980001,2017-01-09,3148.53,3173.14,3147.74,3171.24,1
...,...,...,...,...,...,...,...,...
177,上證指數,980001,2017-09-21,3364.70,3377.88,3356.88,3357.81,0
178,上證指數,980001,2017-09-22,3347.16,3356.45,3334.98,3352.53,0
179,上證指數,980001,2017-09-25,3344.59,3350.96,3334.94,3341.55,0
180,上證指數,980001,2017-09-26,3336.35,3347.16,3332.60,3343.58,1


In [ ]:
import torch
import torch.utils.data as Data
import numpy as np

In [ ]:
class MydDataset(Data.Dataset):
  def __init__(self, seq_length, file_path, train=True, transform=None):
    df = pd.read_excel(file_path)
    df_len = len(df) - 1
    df_len -= (df_len % seq_length)
    total_size = df_len//seq_length
    train_size = int(total_size * 0.7)
    
    f = df.iloc[:(train_size*seq_length), 3:7]
    l = df.iloc[1:(train_size*seq_length)+1, 7:] #label
    data_size = train_size
    if not train:
      f = df.iloc[(train_size*seq_length):df_len, 3:7]
      l = df.iloc[(train_size*seq_length)+1:df_len+1, 7:] #label
      data_size = total_size - train_size
    
    self.features = np.array(f).astype('float32').reshape(data_size, seq_length, 4)
    self.labels = np.array(l).astype('int').reshape(data_size, seq_length, 1)
    self.transform = transform

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    X = self.features[idx]
    y = self.labels[idx]

    if self.transform:
      X = self.transform(X)
      y = self.transform(y)

    return X, y

In [ ]:
def toTensor(x):
  return torch.tensor(x)

In [ ]:
train_dataset = MydDataset(file_path=r'上證指數.xlsx',
                           seq_length=5,
                           train=True,
                           transform=toTensor)

test_dataset = MydDataset(file_path=r'上證指數.xlsx',
                           seq_length=5,
                           train=False,
                           transform=toTensor)


In [ ]:
fetures, labels = train_dataset[2:9]
fetures.shape, labels.shape 

(torch.Size([7, 5, 4]), torch.Size([7, 5, 1]))

In [ ]:
train_loader = Data.DataLoader(dataset=train_dataset, 
                               batch_size=5,
                               shuffle=True)

test_loader = Data.DataLoader(dataset=test_dataset, 
                               batch_size=5,
                               shuffle=False)

NameError: ignored

In [ ]:
for idx, (seqs, labels) in enumerate(train_loader):
  print(seqs.shape)
  seqs = seqs
seqs

torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])


tensor([[[3192.8899, 3193.0601, 3174.3101, 3182.8000],
         [3179.2200, 3207.3101, 3174.7100, 3207.1299],
         [3203.8601, 3215.9500, 3188.7700, 3212.4399],
         [3203.8201, 3219.5200, 3195.2900, 3217.9600],
         [3208.4600, 3223.3401, 3203.2100, 3212.6299]],

        [[3101.2900, 3120.6599, 3100.3899, 3110.0601],
         [3125.3301, 3143.2800, 3111.5601, 3117.1799],
         [3108.4199, 3113.5200, 3097.6799, 3102.6201],
         [3094.2300, 3110.3899, 3081.8501, 3105.5400],
         [3102.1101, 3105.5100, 3084.8301, 3091.6599]],

        [[3167.5701, 3174.5801, 3157.3301, 3161.6699],
         [3156.6899, 3167.0300, 3136.2700, 3136.7500],
         [3133.6001, 3144.9700, 3115.9800, 3119.2900],
         [3116.0801, 3130.5100, 3102.1599, 3112.7600],
         [3104.4900, 3105.1399, 3044.2900, 3103.4299]],

        [[3272.1899, 3286.6699, 3265.7600, 3281.0000],
         [3280.6201, 3295.1899, 3275.0500, 3286.6201],
         [3285.4600, 3285.4600, 3265.0100, 3269.3899],
    

In [ ]:
input_size = 4
hidden_size = 32 # memory_cell
num_layers = 1
num_class = 1 # output_layer

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_class):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.normal = nn.BatchNorm1d(5, affine=True) #batch_size
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # batch_first batch 為第一個維度
    self.fc = nn.Linear(hidden_size, num_class)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    h0 = Variable(torch.zeros(num_layers, x.size(0), hidden_size))
    c0 = Variable(torch.zeros(num_layers, x.size(0), hidden_size))
    out = self.normal(x)
    out, (h, c) = self.lstm(out, (h0, c0))
    out = self.fc(out)
    out = self.sigmoid(out)

    return out

In [ ]:
rnn = RNN(input_size, hidden_size, num_layers, num_class)
rnn

RNN(
  (normal): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(4, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
output = rnn(seqs)
output.size()

torch.Size([5, 5, 1])

In [ ]:
import torch.optim as opt

In [ ]:
loss_func = nn.BCELoss()
optimizer = opt.Adam(rnn.parameters(), lr=0.001)

In [ ]:
for epoch in range(800):
  for idx, (seqs, labels) in enumerate(train_loader):
    seqs = Variable(seqs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = rnn(seqs)
    loss = loss_func(outputs, labels.float()) # crossentropy need float number
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
      print('Epoch: %d, batch: %d, Loss: %.4f'%(epoch+1, idx+1, loss.data))

Epoch: 100, batch: 1, Loss: 0.5538
Epoch: 100, batch: 2, Loss: 0.5621
Epoch: 100, batch: 3, Loss: 0.4573
Epoch: 100, batch: 4, Loss: 0.4468
Epoch: 100, batch: 5, Loss: 0.7158
Epoch: 200, batch: 1, Loss: 0.5121
Epoch: 200, batch: 2, Loss: 0.4541
Epoch: 200, batch: 3, Loss: 0.4747
Epoch: 200, batch: 4, Loss: 0.7025
Epoch: 200, batch: 5, Loss: 0.4312
Epoch: 300, batch: 1, Loss: 0.6983
Epoch: 300, batch: 2, Loss: 0.3692
Epoch: 300, batch: 3, Loss: 0.3496
Epoch: 300, batch: 4, Loss: 0.5334
Epoch: 300, batch: 5, Loss: 0.4219
Epoch: 400, batch: 1, Loss: 0.5182
Epoch: 400, batch: 2, Loss: 0.3651
Epoch: 400, batch: 3, Loss: 0.2913
Epoch: 400, batch: 4, Loss: 0.3024
Epoch: 400, batch: 5, Loss: 0.4160
Epoch: 500, batch: 1, Loss: 0.4135
Epoch: 500, batch: 2, Loss: 0.3847
Epoch: 500, batch: 3, Loss: 0.3823
Epoch: 500, batch: 4, Loss: 0.3325
Epoch: 500, batch: 5, Loss: 0.7146
Epoch: 600, batch: 1, Loss: 0.3791
Epoch: 600, batch: 2, Loss: 0.4165
Epoch: 600, batch: 3, Loss: 0.4541
Epoch: 600, batch: 4

In [ ]:
correct = 0
total = 0
for seqs, labels in test_loader:
  seqs = Variable(seqs)
  outputs = rnn(seqs)
  predicted = outputs.gt(0.5) # bool
  total += (labels.size(0)*labels.size(1))
  correct += (predicted == labels).sum()

print('Acc: %.3f %%' % (100.0 * float(correct)//float(total)))

Acc: 63.000 %
